**Task 09: Data linking**

In [ ]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4/"

In [ ]:
from rdflib import Graph, Namespace, Literal, URIRef
g1 = Graph()
g2 = Graph()
g3 = Graph()
g1.parse(github_storage+"resources/data03.rdf", format="xml")
g2.parse(github_storage+"resources/data04.rdf", format="xml")

Busca individuos en los dos grafos y enlázalos mediante la propiedad OWL:sameAs, inserta estas coincidencias en g3. Consideramos dos individuos iguales si tienen el mismo apodo y nombre de familia. Ten en cuenta que las URI no tienen por qué ser iguales para un mismo individuo en los dos grafos.

**============= SOLUCIÓN PROPUESTA =============**

In [ ]:
from rdflib import XSD, RDF, OWL
from rdflib.plugins.sparql import prepareQuery

g3.bind("owl", OWL, replace=True)

VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")
NS_G1 = Namespace("http://data.three.org#")
NS_G2 = Namespace("http://data.four.org#")

# Descartamos los sujetos que no tienen ambas propiedades en alguno de los dos grafos
# al no hacer OPTIONAL las propiedades

q1 = prepareQuery('''
  SELECT 
    ?subject ?given ?family
	WHERE { 
    ?subject a :Person.
    ?subject vcard:Given ?given.
    ?subject vcard:Family ?family.
  }
  ''',
  initNs = { "vcard": VCARD, "": NS_G1}
)

q2 = prepareQuery('''
  SELECT 
    ?subject
	WHERE { 
    ?subject a :Person.
    ?subject vcard:Given ?given.
    ?subject vcard:Family ?family.
  }
  ''',
  initNs = { "vcard": VCARD, "": NS_G2}
)

for r in g1.query(q1):

  binds = {'?given': r[1], '?family': r[2]}

  same = list(g2.query(q2, initBindings = binds))

  if len(same) == 1:

    g3.add((r[0], OWL.sameAs, same[0][0]))

  
print(g3.serialize(format="ttl").decode("UTF-8"))
